In [265]:
import pandas as pd
import re
import numpy as np

In [269]:
df = pd.read_csv('Shirley_Multiple_Sclerosis_Curation_raw.csv') # read raw dataframe
df.head(10)

,Unnamed: 0,Name,Gene(s),Protein change,Condition(s),Clinical significance (Last reviewed),Review status,Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,AlleleID(s),dbSNP ID,Unnamed: 14,geneID,geneIDs
0,0,"HLA-DRB1, DRB1*1501",HLA-DRB1,NaN,Multiple sclerosis susceptibility 1,"risk factor(Last reviewed: Mar 19, 2018)",no assertion criteria provided,VCV000029757,NaN,NaN,NaN,NaN,29757,38712,NaN,NaN,57679,['3123']
1,1,"IL1RN, IVS2, 86-BP DUP",IL1RN,NaN,Microvascular complications of diabetes 4|Gast...,"Pathogenic, risk factor(Last reviewed: Jul 1, ...",no assertion criteria provided,VCV000014674,NaN,NaN,NaN,NaN,14674,29713,NaN,NaN,57679,['3557']
2,2,"PTPRC, 77C-G",PTPRC,NaN,"Hepatitis c virus, susceptibility to","risk factor(Last reviewed: Jun 3, 2011)",no assertion criteria provided,VCV000014456,NaN,NaN,NaN,NaN,14456,29495,NaN,NaN,57679,['5788']
3,3,"AMER1, DEL",AMER1,NaN,Osteopathia striata with cranial sclerosis,"Pathogenic(Last reviewed: Jan 1, 2010)",no assertion criteria provided,VCV000010706,NaN,NaN,NaN,NaN,10706,25745,NaN,NaN,57679,['139285']
4,4,NM_007375.3(TARDBP):c.198T>C (p.Ala66=),TARDBP,NaN,"Amyotrophic Lateral Sclerosis, Dominant|Amyotr...","Benign/Likely benign(Last reviewed: Dec 29, 2017)","criteria provided, multiple submitters, no con...",VCV000291734,1,11073982,1,11013925,291734,275840,rs61730366,NaN,57679,['23435']
5,5,NM_007375.3(TARDBP):c.859G>A (p.Gly287Ser),TARDBP,G287S,Amyotrophic lateral sclerosis type 10|TARDBP-r...,Conflicting interpretations of pathogenicity(L...,"criteria provided, conflicting interpretations",VCV000021483,1,11082325,1,11022268,21483,34335,rs80356719,NaN,57679,['23435']
6,6,NM_007375.3(TARDBP):c.1098C>G (p.Ala366=),TARDBP,NaN,"Amyotrophic Lateral Sclerosis, Dominant|not sp...","Benign/Likely benign(Last reviewed: Feb 13, 2017)","criteria provided, multiple submitters, no con...",VCV000291740,1,11082564,1,11022507,291740,275852,rs148325203,NaN,57679,['23435']
7,7,NM_007375.3(TARDBP):c.1150G>C (p.Gly384Arg),TARDBP,G384R,TARDBP-related frontotemporal dementia|Amyotro...,"Uncertain significance(Last reviewed: Jul 16, ...","criteria provided, multiple submitters, no con...",VCV000190399,1,11082616,1,11022559,190399,188225,rs797044594,NaN,57679,['23435']
8,8,NM_006610.4(MASP2):c.1479C>T (p.Ser493=),MASP2|TARDBP,NaN,"Amyotrophic Lateral Sclerosis, Dominant|not sp...","Benign(Last reviewed: Jun 14, 2016)","criteria provided, multiple submitters, no con...",VCV000291781,1,11087524,1,11027467,291781,275846,rs1782455,NaN,57679,['10747</GeneID>\\n\\t\\t\\t<strand>-</strand>...
9,9,NM_015967.7(PTPN22):c.1858C>T (p.Arg620Trp),PTPN22|AP4B1-AS1,"R620W, R596W, R565W","Addison disease, susceptibility to|Diabetes me...","risk factor(Last reviewed: Aug 14, 2011)",no assertion criteria provided,VCV000008909,1,114377568,1,113834946,8909,23948,rs2476601,NaN,57679,['26191</GeneID>\\n\\t\\t\\t<strand>-</strand>...


In [270]:
# clean geneID
for i, Id in enumerate(df.geneIDs): 
    geneId = df.geneIDs.iloc[i]
    if len(geneId) > 100:
        df.geneIDs.iloc[i] = re.sub(r'</GeneID>.*<GeneID>', ',', geneId)
    

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [271]:
# check abnormal data
lengths = pd.DataFrame({'len':[len(x) for x in df.geneIDs]})
lengths.len.value_counts()

8     1542
9      215
10     122
7       54
18      17
13       7
19       4
15       3
12       3
2        3
14       2
Name: len, dtype: int64

In [272]:
# drop abnormal data
df = df.drop('Unnamed: 14',axis=1)

#df = df.drop('geneID',axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 16 columns):
Unnamed: 0                               1972 non-null int64
Name                                     1972 non-null object
Gene(s)                                  1972 non-null object
Protein change                           1212 non-null object
Condition(s)                             1972 non-null object
Clinical significance (Last reviewed)    1972 non-null object
Review status                            1972 non-null object
Accession                                1972 non-null object
GRCh37Chromosome                         1968 non-null object
GRCh37Location                           1968 non-null object
GRCh38Chromosome                         1968 non-null object
GRCh38Location                           1968 non-null object
VariationID                              1972 non-null int64
AlleleID(s)                              1972 non-null int64
dbSNP ID                    

In [273]:
# download all gene tissue differentiation expression table from expression Atlas
expression = pd.read_csv('./E-MTAB-5214-query-results.tpms.csv')

In [302]:
tissue_names = expression.columns
# query expression according to gene ID in expression dataframe
urlstem = 'https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5214/Results?geneQuery='

urls = []
top_tissue = []
bad_name = []
for name in df['Gene(s)']:
    try:
        names = name.split('|')[0] # only search the first gene each entry so far

        hit = expression.loc[expression['Gene Name'] == names] # found the gene in expression
        gene = hit.values[0][0] # extract expression levels across >50 tissues
        urls.append(urlstem+str(gene)) # append the url associated with the gene

        s = hit.values[0][2:] 
    
        idx = sorted(range(len(s)), key=lambda k: s[k]) # sort expression level and achieve the top 3 tissue

        top_tissue.append(tissue_names[idx[-3:]]) # query the tissue name according to sorted index
    except:
        top_tissue.append('NA') # if any error happend (gene not found), append 'NA'
        bad_name.append(name)
        print('not found:',name)
    

In [304]:
df['urls'] = urls # add url column


In [305]:
df['GTex_expression'] = top_tissue # add gene expression column

In [306]:
df['GTex_expression'] = df['GTex_expression'].map(lambda x: list(x)) # clean column

In [307]:
df.head(10)

,Unnamed: 0,Name,Gene(s),Protein change,Condition(s),Clinical significance (Last reviewed),Review status,Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,AlleleID(s),dbSNP ID,geneIDs,GTex_expression,urls
0,0,"HLA-DRB1, DRB1*1501",HLA-DRB1,NaN,Multiple sclerosis susceptibility 1,"risk factor(Last reviewed: Mar 19, 2018)",no assertion criteria provided,VCV000029757,NaN,NaN,NaN,NaN,29757,38712,NaN,['3123'],"[skeletal muscle tissue, liver, Brodmann (1909...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
1,1,"IL1RN, IVS2, 86-BP DUP",IL1RN,NaN,Microvascular complications of diabetes 4|Gast...,"Pathogenic, risk factor(Last reviewed: Jul 1, ...",no assertion criteria provided,VCV000014674,NaN,NaN,NaN,NaN,14674,29713,NaN,['3557'],"[subcutaneous adipose tissue, urinary bladder,...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
2,2,"PTPRC, 77C-G",PTPRC,NaN,"Hepatitis c virus, susceptibility to","risk factor(Last reviewed: Jun 3, 2011)",no assertion criteria provided,VCV000014456,NaN,NaN,NaN,NaN,14456,29495,NaN,['5788'],"[Brodmann (1909) area 9, skeletal muscle tissu...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
3,3,"AMER1, DEL",AMER1,NaN,Osteopathia striata with cranial sclerosis,"Pathogenic(Last reviewed: Jan 1, 2010)",no assertion criteria provided,VCV000010706,NaN,NaN,NaN,NaN,10706,25745,NaN,['139285'],"[transverse colon, Brodmann (1909) area 9, min...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
4,4,NM_007375.3(TARDBP):c.198T>C (p.Ala66=),TARDBP,NaN,"Amyotrophic Lateral Sclerosis, Dominant|Amyotr...","Benign/Likely benign(Last reviewed: Dec 29, 2017)","criteria provided, multiple submitters, no con...",VCV000291734,1,11073982,1,11013925,291734,275840,rs61730366,['23435'],"[minor salivary gland, thyroid gland, transver...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
5,5,NM_007375.3(TARDBP):c.859G>A (p.Gly287Ser),TARDBP,G287S,Amyotrophic lateral sclerosis type 10|TARDBP-r...,Conflicting interpretations of pathogenicity(L...,"criteria provided, conflicting interpretations",VCV000021483,1,11082325,1,11022268,21483,34335,rs80356719,['23435'],"[minor salivary gland, thyroid gland, transver...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
6,6,NM_007375.3(TARDBP):c.1098C>G (p.Ala366=),TARDBP,NaN,"Amyotrophic Lateral Sclerosis, Dominant|not sp...","Benign/Likely benign(Last reviewed: Feb 13, 2017)","criteria provided, multiple submitters, no con...",VCV000291740,1,11082564,1,11022507,291740,275852,rs148325203,['23435'],"[minor salivary gland, thyroid gland, transver...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
7,7,NM_007375.3(TARDBP):c.1150G>C (p.Gly384Arg),TARDBP,G384R,TARDBP-related frontotemporal dementia|Amyotro...,"Uncertain significance(Last reviewed: Jul 16, ...","criteria provided, multiple submitters, no con...",VCV000190399,1,11082616,1,11022559,190399,188225,rs797044594,['23435'],"[minor salivary gland, thyroid gland, transver...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
8,8,NM_006610.4(MASP2):c.1479C>T (p.Ser493=),MASP2|TARDBP,NaN,"Amyotrophic Lateral Sclerosis, Dominant|not sp...","Benign(Last reviewed: Jun 14, 2016)","criteria provided, multiple submitters, no con...",VCV000291781,1,11087524,1,11027467,291781,275846,rs1782455,"['10747,23435']","[breast, caudate nucleus, hippocampus proper]",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...
9,9,NM_015967.7(PTPN22):c.1858C>T (p.Arg620Trp),PTPN22|AP4B1-AS1,"R620W, R596W, R565W","Addison disease, susceptibility to|Diabetes me...","risk factor(Last reviewed: Aug 14, 2011)",no assertion criteria provided,VCV000008909,1,114377568,1,113834946,8909,23948,rs2476601,"['26191,100287722']","[aorta, skeletal muscle tissue, Brodmann (1909...",https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5...


In [310]:
df.iloc[0]['urls'] # check url

'https://www.ebi.ac.uk/gxa/experiments/E-MTAB-5214/Results?geneQuery=ENSG00000196126'

In [311]:
# save clean and full dataframe to file
df.to_csv('Shirley_Multiple_Sclerosis_Curation_clean.csv')